In [2]:
from veggideas.load_data import *

Found 15000 files belonging to 15 classes.
Loading training data
Training data successfully loaded ✅


In [3]:
train_data = load_train_data()

Found 15000 files belonging to 15 classes.
Loading training data
Training data successfully loaded ✅


In [7]:
train_data

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 15), dtype=tf.float32, name=None))>

In [8]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,  # Rotate images by 20 degrees
    width_shift_range=0.2,  # Shift images horizontally by 20% of the total width
    height_shift_range=0.2,  # Shift images vertically by 20% of the total height
    shear_range=0.2,  # Apply shear transformation with a shear intensity of 20%
    zoom_range=0.2,  # Apply zoom transformation with a zoom range of 20%
    horizontal_flip=True,  # Flip images horizontally
    fill_mode='nearest'  # Fill newly created pixels during transformations with the nearest value
)

# Create an empty list to store augmented images and labels
augmented_images = []
augmented_labels = []

# Iterate over each batch in the training dataset
for images, labels in train_data:
    # Apply data augmentation to the batch
    augmented_batch = datagen.flow(images, labels, batch_size=images.shape[0])
    augmented_images.extend(augmented_batch[0][0])
    augmented_labels.extend(augmented_batch[0][1])

# Convert the augmented images and labels to TensorFlow tensors
augmented_images = tf.convert_to_tensor(augmented_images)
augmented_labels = tf.convert_to_tensor(augmented_labels)

# Create a new dataset using the augmented images and labels
augmented_train_data = tf.data.Dataset.from_tensor_slices((augmented_images, augmented_labels))


2023-06-06 12:16:16.232836: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [9]:
augmented_train_data

<TensorSliceDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(15,), dtype=tf.float32, name=None))>

In [8]:
from keras import layers

In [3]:
def preprocessing_image():
    resize_and_rescale = tf.keras.Sequential([
        layers.Rescaling(1./255)
    ])
    
    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        ])
    return resize_and_rescale, data_augmentation

In [4]:
import tensorflow as tf
from veggideas.load_data import load_train_data
from veggideas.load_data import load_val_data

train_data = load_train_data()
val_data = load_val_data()

Found 15000 files belonging to 15 classes.
Loading training data
Training data successfully loaded ✅
Found 3000 files belonging to 15 classes.
Loading validation data
Validation data successfully loaded ✅


In [9]:
resize_and_rescale = tf.keras.Sequential([
    layers.Rescaling(1./255)])
    
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    ])

In [10]:
model = tf.keras.Sequential([
  # Add the preprocessing layers you created earlier.
  resize_and_rescale,
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  # Rest of your model.
])